<a href="https://colab.research.google.com/github/praseedm/tracin_implemention_on_ledgar_dataset/blob/main/Find_opponent_%26_proponents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone git repo for data
! git clone https://github.com/praseedm/tracin_implemention_on_ledgar_dataset.git

Cloning into 'tracin_implemention_on_ledgar_dataset'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 48 (delta 19), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [2]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd tracin_implemention_on_ledgar_dataset

/content/tracin_implemention_on_ledgar_dataset


In [4]:
# copying checkpoints 
!cp -r /content/drive/MyDrive/ARU/ML\ Project/train_outputs ./

In [5]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 46.7 MB/s 


In [6]:
import os

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_text as text
import pandas as pd
import json


tf.get_logger().setLevel('ERROR')

In [7]:
!nvidia-smi

Tue Apr 26 20:50:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
tf.__version__

'2.8.0'

## Data Preparation

In [9]:
BATCH_SIZE = 32
SEED = 10
AUTOTUNE = tf.data.AUTOTUNE

In [10]:
def generate_batch_dataset_from_csv(csv_path:str):
  df = pd.read_csv(csv_path)
  dataset = tf.data.Dataset.from_tensor_slices(
      (
          df['text'].values,
          df['label'].values
      )
  )
  print(f"{len(dataset)}\n{dataset.element_spec}")
  return dataset.batch(batch_size=BATCH_SIZE)

In [11]:
train_ds = generate_batch_dataset_from_csv(csv_path='./data/train_data.csv')
test_ds = generate_batch_dataset_from_csv(csv_path='./data/test_data.csv')

32906
(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))
5438
(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


In [12]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
# Load labels 
with open('./data/label_names.json') as rb:
  label_names = json.load(rb)
print(f"Loaded {len(label_names)} labels")
print(f"Labels : {label_names}")

Loaded 25 labels
Labels : ['Amendments', 'Assignments', 'Compliance With Laws', 'Confidentiality', 'Counterparts', 'Entire Agreements', 'Expenses', 'Financial Statements', 'Further Assurances', 'General', 'Governing Laws', 'Indemnifications', 'Insurances', 'Litigations', 'No Conflicts', 'Notices', 'Payments', 'Severability', 'Survival', 'Taxes', 'Terminations', 'Terms', 'Use Of Proceeds', 'Waiver Of Jury Trials', 'Waivers']


## Load checkpoints

In [25]:
checkpoints_root_dir = './train_outputs'
def get_checkpoint_dir(epoch_number:int):
  checkpoint_path = os.path.join(checkpoints_root_dir, f"mymodel_{epoch_number}")
  if os.path.exists(checkpoint_path):
    return checkpoint_path
  raise ValueError(f"{checkpoint_path} doesn't exists")

In [27]:
def load_model(model_path:str):
  print(f"Loading model from {model_path}")
  return tf.keras.models.load_model(model_path)

In [40]:
pred = tf.nn.softmax(model(tf.constant(['Except as otherwise set forth in this Debenture, the Company, for itself and its legal representatives, successors and assigns, expressly waiver'])))

In [41]:
pred

<tf.Tensor: shape=(1, 25), dtype=float32, numpy=
array([[2.2642531e-02, 2.1184969e-01, 3.1306303e-05, 9.8006327e-05,
        2.3974039e-04, 1.6100150e-04, 1.1486170e-03, 8.8653956e-05,
        7.4166362e-04, 3.2395215e-03, 4.8428634e-04, 3.1002457e-03,
        4.6121114e-04, 3.1409060e-04, 9.2119008e-05, 1.1563308e-03,
        1.2305392e-03, 1.6835412e-04, 2.8384614e-04, 1.0147768e-03,
        5.6648382e-04, 6.9146411e-04, 1.2740496e-04, 4.2959419e-03,
        7.4577212e-01]], dtype=float32)>